In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn import metrics
import sklearn
from scipy.stats import rankdata
from bayes_opt import BayesianOptimization

In [2]:
models = [128,192,256,384,512,768]
import glob
# len(glob.glob('../input/submissions/6*.csv'))
sub_paths={}
oof_paths={}
for size in models:
    sub_paths[size] = glob.glob('../input/submissions/6_'+str(size)+'_auc*.csv')
    oof_paths[size] = glob.glob('../input/b6-oofs/oof_6_'+str(size)+'_auc*.csv')

In [3]:
oof_comb = pd.read_csv('../input/b6-oofs/oof_6_128_auc_f0.csv')[0:0] # for base oof db
# oof = pd.read_csv('../input/b6-oofs/oof_6_128_auc_f0.csv')
oof_comb = {} # for base oof db
for size in models:
    oof_comb[size] = pd.read_csv('../input/b6-oofs/oof_6_128_auc_f0.csv')[0:0] # for base oof db
    for i in range(len(oof_paths[size])):
        temp = pd.read_csv(oof_paths[size][i])
        temp['fold'] = int(oof_paths[size][i][-5]) # get fold
        temp.rename(columns={"pred":'pred_'+str(size)}, inplace=True)
        x = temp['pred_'+str(size)].tolist()
        norm_preds = x - np.min(x)
        norm_preds = norm_preds / (np.max(norm_preds)-np.min(norm_preds))
        temp['pred_'+str(size)]=norm_preds
#         temp.drop(columns=[pred',inplace=True)
        oof_comb[size] = oof_comb[size].append(temp)
        oof_comb[size] = oof_comb[size][['image_name','target','pred_'+str(size)]]
oof = oof_comb[128].merge(oof_comb[192]).merge(oof_comb[256]).merge(oof_comb[384]).merge(oof_comb[512]).merge(oof_comb[768])

In [4]:
oof

,image_name,target,pred_128,pred_192,pred_256,pred_384,pred_512,pred_768
0,ISIC_0077735,0.0,0.005929,0.017161,0.000884,0.001280,0.001661,0.003140
1,ISIC_0099474,0.0,0.000329,0.000009,0.000131,0.000600,0.000119,0.000290
2,ISIC_0076995,0.0,0.001013,0.006328,0.000437,0.000372,0.000112,0.000221
3,ISIC_0090279,0.0,0.004264,0.015550,0.001623,0.006492,0.002583,0.007053
4,ISIC_0148465,0.0,0.007720,0.032426,0.009177,0.004473,0.002269,0.002253
...,...,...,...,...,...,...,...,...
32687,ISIC_9972557,0.0,0.001656,0.000830,0.000184,0.000122,0.000288,0.000040
32688,ISIC_9982091,0.0,0.001548,0.000871,0.003306,0.000238,0.082465,0.001488
32689,ISIC_9990676,0.0,0.000513,0.000931,0.000405,0.000160,0.000166,0.000299
32690,ISIC_9995166,0.0,0.267033,0.280448,0.339269,0.364587,0.585901,0.881504


In [5]:
predictions={}
models=[128,192,256,384,512,768]
for size in models:
    predictions[size] = [0]*10982
    for i, path in enumerate(sub_paths[size]):    
        x = pd.read_csv(path)['target'].tolist()
        norm_preds = x - np.min(x)
        norm_preds = norm_preds / (np.max(norm_preds)-np.min(norm_preds))
#         print(norm_preds)
        predictions[size]=[sum(x) for x in zip(predictions[size],norm_preds)]
    predictions[size] = np.array(predictions[size])/len(sub_paths[size])

In [6]:
for size in models:
    print(metrics.roc_auc_score(oof['target'], oof['pred_'+str(size)]),size)

0.8949648677235178 128
0.8829082060501089 192
0.9248504448130145 256
0.9196269545007151 384
0.9289694937810116 512
0.9360367391702975 768


In [7]:
oof

,image_name,target,pred_128,pred_192,pred_256,pred_384,pred_512,pred_768
0,ISIC_0077735,0.0,0.005929,0.017161,0.000884,0.001280,0.001661,0.003140
1,ISIC_0099474,0.0,0.000329,0.000009,0.000131,0.000600,0.000119,0.000290
2,ISIC_0076995,0.0,0.001013,0.006328,0.000437,0.000372,0.000112,0.000221
3,ISIC_0090279,0.0,0.004264,0.015550,0.001623,0.006492,0.002583,0.007053
4,ISIC_0148465,0.0,0.007720,0.032426,0.009177,0.004473,0.002269,0.002253
...,...,...,...,...,...,...,...,...
32687,ISIC_9972557,0.0,0.001656,0.000830,0.000184,0.000122,0.000288,0.000040
32688,ISIC_9982091,0.0,0.001548,0.000871,0.003306,0.000238,0.082465,0.001488
32689,ISIC_9990676,0.0,0.000513,0.000931,0.000405,0.000160,0.000166,0.000299
32690,ISIC_9995166,0.0,0.267033,0.280448,0.339269,0.364587,0.585901,0.881504


In [8]:
train = pd.read_csv('../input/melanoma-128x128/train.csv')[['image_name','patient_code']]

In [9]:
oof = oof.merge(train,on=['image_name'])

In [10]:
oof

,image_name,target,pred_128,pred_192,pred_256,pred_384,pred_512,pred_768,patient_code
0,ISIC_0077735,0.0,0.005929,0.017161,0.000884,0.001280,0.001661,0.003140,14
1,ISIC_0099474,0.0,0.000329,0.000009,0.000131,0.000600,0.000119,0.000290,56
2,ISIC_0076995,0.0,0.001013,0.006328,0.000437,0.000372,0.000112,0.000221,12
3,ISIC_0090279,0.0,0.004264,0.015550,0.001623,0.006492,0.002583,0.007053,42
4,ISIC_0148465,0.0,0.007720,0.032426,0.009177,0.004473,0.002269,0.002253,74
...,...,...,...,...,...,...,...,...,...
32687,ISIC_9972557,0.0,0.001656,0.000830,0.000184,0.000122,0.000288,0.000040,2005
32688,ISIC_9982091,0.0,0.001548,0.000871,0.003306,0.000238,0.082465,0.001488,529
32689,ISIC_9990676,0.0,0.000513,0.000931,0.000405,0.000160,0.000166,0.000299,1873
32690,ISIC_9995166,0.0,0.267033,0.280448,0.339269,0.364587,0.585901,0.881504,1419


In [11]:
models

[128, 192, 256, 384, 512, 768]

In [12]:
from tqdm import tqdm

new_oof = oof
for size in models:
    db = oof
    db_ = db[0:0]
    for p in tqdm(range(2056)):
        db = oof.query('patient_code=='+str(p))
        tar_min = db['pred_'+str(size)].min()
        #         print(tar_min)
        db['pred_'+str(size)+'d'] = db['pred_'+str(size)] - tar_min
        db_ = db_.append(db)
    new_oof = new_oof.merge(db_,on=['image_name','target','pred_128','pred_192','pred_256','pred_384','pred_512','pred_768','patient_code'])

  0%|          | 0/2056 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 2056/2056 [00:19<00:00, 105.36it/s]


In [13]:
new_oof

,image_name,target,pred_128,pred_192,pred_256,pred_384,pred_512,pred_768,patient_code,pred_128d,pred_192d,pred_256d,pred_384d,pred_512d,pred_768d
0,ISIC_0077735,0.0,0.005929,0.017161,0.000884,0.001280,0.001661,0.003140,14,0.005501,0.017094,0.000744,0.001184,0.001615,0.003129
1,ISIC_0099474,0.0,0.000329,0.000009,0.000131,0.000600,0.000119,0.000290,56,0.000315,0.000009,0.000109,0.000589,0.000119,0.000289
2,ISIC_0076995,0.0,0.001013,0.006328,0.000437,0.000372,0.000112,0.000221,12,0.000287,0.000000,0.000111,0.000053,0.000000,0.000000
3,ISIC_0090279,0.0,0.004264,0.015550,0.001623,0.006492,0.002583,0.007053,42,0.004127,0.015537,0.001441,0.006317,0.002558,0.007052
4,ISIC_0148465,0.0,0.007720,0.032426,0.009177,0.004473,0.002269,0.002253,74,0.007605,0.032425,0.009119,0.004426,0.002252,0.002237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32687,ISIC_9972557,0.0,0.001656,0.000830,0.000184,0.000122,0.000288,0.000040,2005,0.001152,0.000745,0.000127,0.000046,0.000270,0.000039
32688,ISIC_9982091,0.0,0.001548,0.000871,0.003306,0.000238,0.082465,0.001488,529,0.001344,0.000762,0.003144,0.000137,0.082145,0.001486
32689,ISIC_9990676,0.0,0.000513,0.000931,0.000405,0.000160,0.000166,0.000299,1873,0.000416,0.000907,0.000325,0.000000,0.000000,0.000000
32690,ISIC_9995166,0.0,0.267033,0.280448,0.339269,0.364587,0.585901,0.881504,1419,0.206064,0.227173,0.314631,0.352234,0.529090,0.841797


In [14]:
from sklearn.linear_model import LogisticRegression
X = new_oof[['pred_128','pred_192','pred_256','pred_384','pred_512','pred_768',
            'pred_128d','pred_192d','pred_256d','pred_384d','pred_512d','pred_768d']].to_numpy()
Y = new_oof['target'].values
meta_model = LogisticRegression(solver='liblinear')
meta_model.fit(X, Y)
Y_p = meta_model.predict_proba(X)
Y_pred=[]
for y in Y_p:
    Y_pred.append(y[1])
Y_pred = np.array(Y_pred)
metrics.roc_auc_score(Y, Y_pred)

0.9431893703912488

In [15]:
X_sub = pd.DataFrame(np.transpose(np.array([predictions[128],predictions[192],predictions[256],predictions[384],predictions[512],predictions[768]])),columns=['pred_128','pred_192','pred_256','pred_384','pred_512','pred_768'])

In [16]:
meta_test = pd.read_csv('../input/freshstart128/test.csv')
meta_test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')[['image_name']].merge(meta_test,on='image_name')[['image_name', 'patient_id']]
img_name = meta_test['image_name'].tolist()

In [17]:
X_sub['image_name'] = img_name
X_sub = X_sub.merge(meta_test, on ='image_name')

In [18]:
X_sub

,pred_128,pred_192,pred_256,pred_384,pred_512,pred_768,image_name,patient_id
0,0.002321,0.000505,0.001207,0.000843,0.000840,0.001460,ISIC_0052060,100129
1,0.001134,0.004287,0.000292,0.000620,0.000352,0.000042,ISIC_0052349,100185
2,0.000688,0.000125,0.000382,0.000451,0.000909,0.000911,ISIC_0058510,100175
3,0.000660,0.000546,0.000732,0.000243,0.000517,0.000950,ISIC_0073313,100429
4,0.011500,0.018426,0.002330,0.002361,0.007074,0.060007,ISIC_0073502,100021
...,...,...,...,...,...,...,...,...
10977,0.011137,0.012813,0.000854,0.003131,0.002662,0.015549,ISIC_9992485,100195
10978,0.055366,0.173387,0.015413,0.017118,0.047121,0.160123,ISIC_9996992,100037
10979,0.070029,0.114137,0.047680,0.103786,0.164000,0.566856,ISIC_9997917,100334
10980,0.009396,0.012782,0.004316,0.010774,0.005116,0.006746,ISIC_9998234,100502


In [19]:
from tqdm import tqdm

new_oof = oof
for size in models:
    db = oof
    db_ = db[0:0]
    for p in tqdm(range(2056)):
        db = oof.query('patient_code=='+str(p))
        tar_min = db['pred_'+str(size)].min()
        #         print(tar_min)
        db['pred_'+str(size)+'d'] = db['pred_'+str(size)] - tar_min
        db_ = db_.append(db)
    new_oof = new_oof.merge(db_,on=['image_name','target','pred_128','pred_192','pred_256','pred_384','pred_512','pred_768','patient_code'])

  0%|          | 0/2056 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 2056/2056 [00:19<00:00, 104.80it/s]


In [20]:
new_X_sub = X_sub
for size in models:
    db = X_sub
    db_ = db[0:0]
    for p in tqdm(range(690)):
        db = X_sub.query('patient_id=='+str(p+100000))
        tar_min = db['pred_'+str(size)].min()
        #         print(tar_min)
        db['pred_'+str(size)+'d'] = db['pred_'+str(size)] - tar_min
        db_ = db_.append(db)
    new_X_sub = new_X_sub.merge(db_,on=['image_name','pred_128','pred_192','pred_256','pred_384','pred_512','pred_768','patient_id'])

  0%|          | 0/690 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
100%|██████████| 690/690 [00:05<00:00, 129.21it/s]


In [21]:
new_X_sub

,pred_128,pred_192,pred_256,pred_384,pred_512,pred_768,image_name,patient_id,pred_128d,pred_192d,pred_256d,pred_384d,pred_512d,pred_768d
0,0.002321,0.000505,0.001207,0.000843,0.000840,0.001460,ISIC_0052060,100129,0.002230,0.000485,0.001174,0.000830,0.000834,0.001416
1,0.001134,0.004287,0.000292,0.000620,0.000352,0.000042,ISIC_0052349,100185,0.001024,0.004234,0.000269,0.000561,0.000348,0.000042
2,0.000688,0.000125,0.000382,0.000451,0.000909,0.000911,ISIC_0058510,100175,0.000374,0.000006,0.000291,0.000410,0.000838,0.000897
3,0.000660,0.000546,0.000732,0.000243,0.000517,0.000950,ISIC_0073313,100429,0.000297,0.000517,0.000602,0.000148,0.000472,0.000935
4,0.011500,0.018426,0.002330,0.002361,0.007074,0.060007,ISIC_0073502,100021,0.010247,0.017996,0.002175,0.002258,0.007012,0.058979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10977,0.011137,0.012813,0.000854,0.003131,0.002662,0.015549,ISIC_9992485,100195,0.006476,0.008047,0.000185,0.000379,0.001444,0.015491
10978,0.055366,0.173387,0.015413,0.017118,0.047121,0.160123,ISIC_9996992,100037,0.055242,0.173369,0.015401,0.017099,0.047094,0.160098
10979,0.070029,0.114137,0.047680,0.103786,0.164000,0.566856,ISIC_9997917,100334,0.063029,0.110420,0.047390,0.103557,0.163784,0.565964
10980,0.009396,0.012782,0.004316,0.010774,0.005116,0.006746,ISIC_9998234,100502,0.008096,0.011751,0.003554,0.010394,0.004449,0.006595


In [22]:
X_sub = new_X_sub[['pred_128','pred_192','pred_256','pred_384','pred_512','pred_768',
            'pred_128d','pred_192d','pred_256d','pred_384d','pred_512d','pred_768d']]

In [23]:
Y_p_sub = meta_model.predict_proba(X_sub)
Y_pred_sub=[]
for y in Y_p_sub:
    Y_pred_sub.append(y[1])
Y_pred_sub = np.array(Y_pred_sub)

In [24]:
Y_pred_sub

array([0.0032866 , 0.00327207, 0.00327912, ..., 0.03063438, 0.00340295,
       0.09019694])

In [25]:
len(Y_pred_sub)

10982

In [26]:
sub = pd.read_csv('../input/submissions/6_384_auc_f0_submission.csv')[['image_name']]
sub['target']=Y_pred_sub
# sub['target_n']=Y_pred_sub-np./0.7322306285432676

In [27]:
sub.to_csv('submission.csv', index=False)

In [28]:
sub['target']

0        0.003287
1        0.003272
2        0.003279
3        0.003279
4        0.004054
           ...   
10977    0.003482
10978    0.006594
10979    0.030634
10980    0.003403
10981    0.090197
Name: target, Length: 10982, dtype: float64